### Load packages

In [1]:

from clease.tools import reconfigure
from clease import  NewStructures,Evaluate
from clease.settings import Concentration, CECrystal, CEBulk
from clease.tools import reconfigure
from clease.regression import PhysicalRidge,LinearRegression
from clease.basis_function import BinaryLinear
from clease.regression.physical_ridge import random_cv_hyper_opt
import logging
logging.basicConfig(level=logging.INFO)
from clease.corr_func import CorrFunction
import clease.plot_post_process as pp

import json# from tqdm import tqdm
import itertools
from ase.visualize import view
import shutil
from ase.io import read
import numpy as np
from ase.db import connect
import matplotlib.pyplot as plt
import os 

### Set lattice parameters from cif file

In [ ]:
# Define the structure from the cif file (cif_files/Na2Fe2SO4.cif)

Fe1Na1 = (0.26987, 0.34210, 0.35406)

Na2 = (0, 0.73305, 0.25)
Na3 = (0.4891, 0.0046, 0.0421)
Na4 = (-0.0072, 0.0113, 0.2839)

S1 = (0.5, 0.2235, 0.75)
S2 = (0.23745, 0.10265, 0.1300)
O1 = (0.08512, 0.34528, 0.22064)
O2 = (0.05370, 0.20951, 0.45382)
O3 = (0.23398, 0.16965, 0.31316)
O4 = (0.18027, 0.00395, 0.12336)
O5 = (0.32586, 0.34197, 0.08521)
O6 = (0.35966, 0.08711, 0.17191)

cellpar = [12.64974,12.76248,6.508625,90,115.5396,90] # Cell parameters, a, b, c, alpha, beta, gamma



### Initialize the cluster expansion object

In [32]:
# Define the database file, which will store the structures generated
db_name = 'alluadite_all_Fe.db'

# Define the concentration of the system all information given in the cif file. X represent vacant sites
conc = Concentration(basis_elements = [
                                       ['Na','X'], # multiplicity factor of 2
                                       ['Na','X'], # multiplicity factor of 4 
                                       ['Na','X'], # multiplicity factor of 4
                                       
                                       ['Fe','Na','X'], # multiplicity factor of 4
                                       
                                       ['S'], # multiplicity factor of 2
                                       ['S'], # multiplicity factor of 4
                                       
                                       ['O'], # multiplicity factor of 4
                                       ['O'], # multiplicity factor of 4
                                       ['O'], # multiplicity factor of 4
                                       ['O'], # multiplicity factor of 4
                                       ['O'], # multiplicity factor of 4
                                       ['O']  # multiplicity factor of 4
                                       ],
                    grouped_basis=[[0,1,2], #  multiplicity factor of 10 (2+4+4)
                                   [3],     # multiplicity factor of 4
                                   [4,5],   # multiplicity factor of 6
                                   [6,7,8,9,10,11]] # multiplicity factor of 24
                    ) 
# Max Na concentration: Na10Fe7(SO4)12
# Min Na concentration: Na6Fe7(SO4)12
# Start with Na10Fe7(SO4)12 --> 10 Na, 7 Fe, 12 S, 48 O for 2 times unit cell
#                                5 Na, 3.5 Fe, 6 S, 24 O for unit cell

# Define the equality constraints for the system
conc.A_eq = np.array([#Na, X, Fe, Na, X, S, O
                      [1, 1, 0, 0, 0, 0, 0], # 10
                      [0, 0, 1, 1, 1, 0, 0], # 4    
                      [0, 0, 0, 0, 0, 1, 0], # 6
                      [0, 0, 0, 0, 0, 0, 1], # 24
                      [0, 0, 8, 0, 0, 0, 0], # Fe (3.5 out of 4 sites)
                      ]   
)                   

# all N sites in primitive = 5 --> 
conc.b_eq = np.array([1,
                      1,
                      1,
                      1,
                      7,
                      ]) 

# Define the inequality constraints
conc.A_lb =np.array([[-10, 0, 0, -4, 0, 0, 0], # Less than 5 Na in unit cell (or 10 in 2x1x1 cell)
                     [10, 0, 0, 4, 0, 0, 0],   # More than 3 Na in unit cell (or 6 in 2x1x1)
                      ]   )
conc.b_lb = np.array([-5, 1.5])


setting = CECrystal(cellpar=cellpar,
                   basis=[Na2,Na3,Na4,Fe1Na1, S1,S2,O1,O2,O3,O4,O5,O6],
                   concentration=conc,
                   spacegroup=15,
                   size=[(2,0,0),(0,1,0),(0,0,1)],
                   db_name=db_name,
                   max_cluster_dia=[5.0, 5.0])
basis_func = ['polynomial', 'trigonometric','binaryLinear']
setting.basis_func_type=basis_func[0] #: polynomial, trigonometric and binaryLinear.

[[1 1 0 0 0 0 0]
 [0 0 1 1 1 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]]
[1 1 1 1]
[[1 1 0 0 0 0 0]
 [0 0 1 1 1 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]
 [0 0 8 0 0 0 0]]
[1 1 1 1 7]


### Generate structures using random sampling

In [3]:
#### Generating Random structures ####
## The samples generated are stored in the database file defined above
# One will then use the workflow/relaxation to relax the structures

sampling = 60 # Number of structures to generate
ns = NewStructures(setting, struct_per_gen=sampling)

count=1
while count !=sampling:
    try:
        ns.generate_one_random_structure()
        count+=1
    except:
        pass

INFO:clease.structgen.new_struct:Generating 100 random structures (generation: 0, struct_per_gen=100, 0 present).
INFO:clease.structgen.new_struct:Succesfully generated 100 random structures.
